In [1]:
from flair.models import TextClassifier
from flair.data import Sentence

import numpy as np
import pandas as pd
#import html
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from emot.emo_unicode import UNICODE_EMOJI, EMOTICONS_EMO

# from sklearn.model_selection import train_test_split
# import evaluate

# import transformers
# from transformers import AutoTokenizer
# from transformers import TFAutoModelForSequenceClassification
# import tensorflow as tf

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\mhatr\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
dfSentiment = pd.read_csv('TrainingData.csv',nrows=10000)
#dfSentiment = dfSentiment.sample(random_state=0, n=10000)
dfSentiment.reset_index(drop=True, inplace=True)

In [3]:
dfSentiment.head()

,data,sentiment
0,Gas by my house hit $3.39!!!! I'm going to Cha...,2
1,Iranian general says Israel's Iron Dome can't ...,0
2,with J Davlar 11th. Main rivals are team Polan...,2
3,"Talking about ACT's && SAT's, deciding where I...",0
4,"They may have a SuperBowl in Dallas, but Dalla...",0


In [5]:
dfSentiment.sentiment.value_counts()

1    3691
2    2834
0    1064
Name: sentiment, dtype: int64

In [ ]:
text = dfSentiment.data[0]
text

In [ ]:
# ###############     Regex defination       ################################

# regex to perform operations
html_re = re.compile('<.*?>') # html links
email = re.compile(r'[\w\.-]+@[\w\.-]+') # emails
punctuation_token=RegexpTokenizer(r'\w+') # punctuation
#non_word_pattern = re.compile(r'[^\w\s]+') # &amp; to & converstion.

# creating stop words regex
cachedStopWords = set(stopwords.words("english"))
#add custom words
cachedStopWords.update(('and','I','A','http','And','So','arnt','This','When','It','many','Many','so','cant','Yes','yes','No','no','These','these','mailto','regards','ayanna','like','email'))
# Define regex pattern to match stop words
stop_words_pattern = re.compile(r'\b(' + r'|'.join(cachedStopWords) + r')\b', flags=re.IGNORECASE)


# creating regex to remove emoji
emoji_pattern = re.compile("["
    u"\U0001F600-\U0001F64F"  # emoticons
    u"\U0001F300-\U0001F5FF"  # symbols & pictographs
    u"\U0001F680-\U0001F6FF"  # transport & map symbols
    u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
    u"\U00002702-\U000027B0"  # other miscellaneous symbols
    u"\U000024C2-\U0001F251" 
    u"\U0001F910-\U0001F96B"  # new emojis introduced in Unicode 9.0
    u"\U0001F980-\U0001F991"  # new emojis introduced in Unicode 10.0
    u"\U0001F9C0"             # new emojis introduced in Unicode 11.0
    "]+", flags=re.UNICODE)

# ###############################################################################

In [ ]:
def clean_text(text):
    
    # remove retweet username and tweeted at @username
    text = re.sub('(@[A-Za-z]+[A-Za-z0-9-_]+)', '', text) 
    
    # remove links
    text = re.sub(r'http\S+', '', text) # remove http links
    text = re.sub(r'bit.ly/\S+', '', text) # remove bitly links
    text = text.strip('[link]') # remove [links]

    # remove html
    text = html_re.sub(r'',text)

    # remove non ascii character
    text = re.sub(r'[^\x00-\x7F]+', '', text)

    # lower the text
    text = text.lower()

    # remove email address
    text = email.sub(r'',text)

    # remove punctuation
    text = punctuation_token.tokenize(text)
    text= " ".join(text)
    
    #remove stopwords
    text = stop_words_pattern.sub('', text)

    # remove Special characters
    text = re.sub('([_]+)', "", text)
    
    # remove emoji
    text = emoji_pattern.sub(r"", text)
    
    # # find hashtags in post
    # text = re.findall(r"#(\w+)", text)
    
    return ' '.join(text.split())

In [ ]:
dfSentiment.data = dfSentiment.data.apply(clean_text)

In [ ]:
# load the pre-trained sentiment classification model
classifier = TextClassifier.load('en-sentiment')

In [ ]:
sentences = [Sentence(i) for i in dfSentiment.data[:100]]

label = []
score = []
for sentence in sentences:
    classifier.predict(sentence)
    label.append(sentence.labels[0].value)
    score.append(sentence.labels[0].score)
    